In [1]:
# Importing necessary libraries
import numpy as np
import librosa
import os

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, SimpleRNN, BatchNormalization
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, EarlyStopping
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
# Define function to extract features from audio
def extract_features(file_path):
    try:
        audio, _ = librosa.load(file_path, res_type='kaiser_fast', duration=30) 
        mfccs = librosa.feature.mfcc(y=audio, sr=22050, n_mfcc=13)
        mfccs_processed = np.mean(mfccs.T,axis=0)
        
    except Exception as e:
        print("Error encountered while parsing file: ", file_path)
        return None 
     
    return mfccs_processed

In [3]:
# Define function to load data and extract features
def load_data(data_path):
    labels = []
    features = []
    
    for folder in os.listdir(data_path):
        genre_folder = os.path.join(data_path, folder)
        for file in os.listdir(genre_folder):
            file_path = os.path.join(genre_folder, file)
            feature = extract_features(file_path)
            if feature is not None:
                features.append(feature)
                labels.append(folder)
                
    return np.array(features), np.array(labels)

In [4]:
# Load data
data_path = 'Data/genres_original'  # Update with your dataset path
features, labels = load_data(data_path)

# Encode labels
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)
labels_categorical = to_categorical(encoded_labels)

C:\Users\JLCarunungan\AppData\Local\Temp\ipykernel_12780\2273535198.py:4: UserWarning: PySoundFile failed. Trying audioread instead.
  audio, _ = librosa.load(file_path, res_type='kaiser_fast', duration=30)
c:\Users\JLCarunungan\AppData\Local\Programs\Python\Python312\Lib\site-packages\librosa\core\audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


Error encountered while parsing file:  Data/genres_original\jazz\jazz.00054.wav


In [5]:
features.shape

(999, 13)

In [6]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels_categorical, test_size=0.2, random_state=42)

In [7]:
X_train.shape[0]

799

In [8]:
# Reshape input data to have the appropriate shape for RNN
X_train_reshaped = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)

In [ ]:
#V_1
# Define the model 
model = Sequential()
model.add(SimpleRNN(units=128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
model.add(Dropout(0.2))  # Adjust dropout rate
model.add(BatchNormalization())  # Add BatchNormalization layer
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

In [ ]:
#V_2
# Define the model
model = Sequential()
model.add(SimpleRNN(units=128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
model.add(Dropout(0.2))  # Adjust dropout rate
model.add(BatchNormalization())  # Add BatchNormalization layer
model.add(SimpleRNN(units=64, return_sequences=True))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(SimpleRNN(units=32))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))

In [9]:
#v_3 HyperTesting

# Define a function to create the model
def create_model(learning_rate=0.001, dropout_rate=0.2):
    model = Sequential()
    model.add(SimpleRNN(units=128, input_shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]), return_sequences=True))
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())
    model.add(SimpleRNN(units=64, return_sequences=True))
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())
    model.add(SimpleRNN(units=32))
    model.add(Dropout(dropout_rate))  # Add dropout layer
    model.add(BatchNormalization())
    model.add(Dense(units=len(label_encoder.classes_), activation='softmax'))
    
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Initialize variables to store best score and parameters
best_score = 0
best_params = {}

# Define lists of hyperparameters to search over
learning_rates = [0.001, 0.01, 0.1, 0.2, 0.25, 0.3]
dropout_rates = [0.1, 0.2, 0.3]
batch_sizes = [16, 32, 64, 128, 256]
epochs_list = [10, 20, 30, 40, 50]

# Iterate over all combinations of hyperparameters
for lr in learning_rates:
    for dropout_rate in dropout_rates:
        for batch_size in batch_sizes:
            for epochs in epochs_list:
                # Create model with current hyperparameters
                model = create_model(learning_rate=lr, dropout_rate=dropout_rate)
                
                # Train the model
                history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2, verbose=0)
                
                # Evaluate the model on validation data
                _, accuracy = model.evaluate(X_test, y_test, verbose=0)
                
                # Store the accuracy and parameters
                params = {'learning_rate': lr, 'dropout_rate': dropout_rate, 'batch_size': batch_size, 'epochs': epochs}
                print("Accuracy:", accuracy, "Params:", params)
                
                # Check if current model has the best score
                if accuracy > best_score:
                    best_score = accuracy
                    best_params = params

# Print the best score and parameters
print("Best Mean Test Accuracy:", best_score)
print("Best Parameters:", best_params)

c:\Users\JLCarunungan\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Accuracy: 0.4350000023841858 Params: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batch_size': 16, 'epochs': 30}
Accuracy: 0.375 Params: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batch_size': 16, 'epochs': 40}
Accuracy: 0.38499999046325684 Params: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batch_size': 16, 'epochs': 50}
Accuracy: 0.4749999940395355 Params: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batch_size': 16, 'epochs': 60}
Accuracy: 0.38499999046325684 Params: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batch_size': 32, 'epochs': 30}
Accuracy: 0.4050000011920929 Params: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batch_size': 32, 'epochs': 40}
Accuracy: 0.41999998688697815 Params: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batch_size': 32, 'epochs': 50}
Accuracy: 0.42500001192092896 Params: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batch_size': 32, 'epochs': 60}
Accuracy: 0.36500000953674316 Params: {'learning_rate': 0.001, 'dropout_rate': 0.1, 'batc

In [ ]:
# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
#checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test))

In [ ]:
# Evaluate model
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Save the model
#model.save('music_genre_classifier_rnn.keras')
#print("Model saved as 'music_genre_classifier_rnn.keras'")